In [1]:
import html
import folium
import pandas as pd
from geolocate import geocode

In [2]:
get_loc = lambda x: (x[0]['geometry']['location']['lat'], x[0]['geometry']['location']['lng'],)

df = pd.read_excel('listings.xlsx')
listings = df.to_dict(orient='records')
addresses = [", ".join([row.address, row.city, "NJ"]) for i, row in df.iterrows()]
geocoded_addresses = [geocode(x) for x in addresses]
locations = [get_loc(x) for x in geocoded_addresses]

home = get_loc(geocode('115 Dumont Ave. Clifton, NJ'))

In [3]:
m = folium.Map(location=home, zoom_start=12)

folium.Marker(home, tooltip=f'<i>115 Dumont Ave. Clifton, NJ</i>', icon=folium.Icon(color='green')).add_to(m)

for l, address, location in zip(listings, addresses, locations):
    id, rating, comments = l['id'], l['rating'], l['comments']
    if str(id).startswith("3"):
        url = f"https://gsmls.herokuapp.com/properties/{id}"
    else:
        url = f"http://www.njmls.com/listings/index.cfm?action=dsp.info&mlsnum={id}"
    comments = '<br/>'.join([html.escape(x) for x in comments.split('\n')])
    popup = f"""
    Address: {address}<br/>
    MLS: <a target="_blank" href="{url}">{id}</a><br/>
    Rating: {rating}<br/>
    Comments:<br/>
    {comments}
    """
    folium.Marker(location, popup=f'<i>{popup}</i>', tooltip=f'<i>{address} - {rating}</i>').add_to(m)

m.save('map.html')